<a href="https://colab.research.google.com/github/juan026/EVA3D/blob/main/ComputerGraphics_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup Everything

!nvidia-smi
!git clone https://github.com/juan026/EVA3D.git

!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install fvcore iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
!pip install -r EVA3D/requirements.txt

Wed Apr 12 06:02:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# @title Download Models
%cd EVA3D
from download_models import download_pretrained_models
download_pretrained_models()

/content/EVA3D


100%|██████████| 160M/160M [00:02<00:00, 60.1MB/s]


100%|██████████| 160M/160M [00:01<00:00, 110MB/s]


100%|██████████| 160M/160M [00:02<00:00, 75.1MB/s]


100%|██████████| 158M/158M [00:01<00:00, 110MB/s]


Register and download SMPL models [here](https://smpl.is.tue.mpg.de/). Put the downloaded models in the folder smpl_models. Only the neutral one is needed. The folder structure should look like

```
./
├── ...
└── smpl_models/
    ├── smpl/
        └── SMPL_NEUTRAL.pkl
```

In [3]:
# @title Download SMPL NEUTRAL model
!ls

!mkdir smpl_models
%cd smpl_models
!mkdir smpl
%cd smpl
!ls
!gdown 1DTiAXbVnCRopZ-LObAqKnT-9rv-aySc5
!ls

assets		  download_datasets.py	model.py	  scripts
augment.py	  download_models.py	notebook	  smpl_models
calculate_fid.py  eva3d_aist.py		op		  smpl_utils.py
checkpoint	  eva3d_deepfashion.py	options.py	  torch_utils
dataset.py	  generation_demo.py	__pycache__	  train_deepfashion.py
distributed.py	  LICENSE		README.md	  utils.py
dnnlib		  losses.py		requirements.txt  volume_renderer.py
mkdir: cannot create directory ‘smpl_models’: File exists
/content/EVA3D/smpl_models
mkdir: cannot create directory ‘smpl’: File exists
/content/EVA3D/smpl_models/smpl
SMPL_NEUTRAL.pkl
Downloading...
From: https://drive.google.com/uc?id=1DTiAXbVnCRopZ-LObAqKnT-9rv-aySc5
To: /content/EVA3D/smpl_models/smpl/SMPL_NEUTRAL.pkl
100% 39.0M/39.0M [00:00<00:00, 247MB/s]
SMPL_NEUTRAL.pkl


In [4]:
# @title Download dataset for training
%cd ../..
!ls
!python download_datasets.py

Streaming output truncated to the last 5000 lines.
  inflating: datasets/DeepFashion/images/WOMEN-Dresses-id_00000481-03_4_full.png  
  inflating: datasets/DeepFashion/images/MEN-Jackets_Vests-id_00004606-01_4_full.png  
  inflating: datasets/DeepFashion/images/WOMEN-Dresses-id_00002682-03_1_front.png  
  inflating: datasets/DeepFashion/images/WOMEN-Dresses-id_00004354-02_2_side.png  
  inflating: datasets/DeepFashion/images/WOMEN-Tees_Tanks-id_00000212-01_4_full.png  
  inflating: datasets/DeepFashion/images/WOMEN-Pants-id_00004070-05_1_front.png  
  inflating: datasets/DeepFashion/images/WOMEN-Sweaters-id_00003632-04_4_full.png  
  inflating: datasets/DeepFashion/images/MEN-Sweatshirts_Hoodies-id_00006478-01_4_full.png  
  inflating: datasets/DeepFashion/images/WOMEN-Blouses_Shirts-id_00000727-04_7_additional.png  
  inflating: datasets/DeepFashion/images/MEN-Sweatshirts_Hoodies-id_00002533-01_4_full.png  
  inflating: datasets/DeepFashion/images/WOMEN-Blouses_Shirts-id_00004111-06_7

In [ ]:
# @title Train a new model using deepfashion dataset
!python train_deepfashion.py --batch 8 --chunk 1 --expname train_deepfashion_512x256 --dataset_path datasets/DeepFashion --depth 5 --width 128 --style_dim 128 --renderer_spatial_output_dim 512 256 --input_ch_views 3 --white_bg --r1 300 --voxhuman_name eva3d_deepfashion --random_flip --eikonal_lambda 0.5 --small_aug --iter 1000 --adjust_gamma --gamma_lb 20 --min_surf_lambda 1.5 --deltasdf --gaussian_weighted_sampler --sampler_std 15 --N_samples 14


image num: 8037
smpl num: 8037
  0% 0/1000 [00:00<?, ?it/s]Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
d: 1.3860; g: 0.6333; r1: 300.0 0.0000; eik: 0.2846; surf: 0.0022; augp: 0.6000; beta: 0.1000:   0% 0/1000 [00:16<?, ?it/s]Successfully saved checkpoint for iteration 0.
d: 1.0797; g: 0.9187; r1: 300.0 0.0068; eik: 0.1605; surf: 0.0023; augp: 0.6000; beta: 0.0997:   1% 12/1000 [03:16<4:29:43, 16.38s/it]